# 27. 用列表推到取代map与filter

In [12]:
# 使用map完成一个列表的平方
a = range(0,10,1)
squares_m = list(map(lambda x: x**2, a)) # map函数生成迭代器，需要使用list将其进行生成

In [13]:
squares_l = [x**2 for x in a]
print(f'列表推导：{squares_l}\n'
      f'map推导{squares_m}')

列表推导：[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
map推导[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [14]:
a

range(0, 10)

In [15]:
# 但是在进行条件筛选时列表更方便
even_squares = [x**2 for x in a if x % 2 ==0]
even_squares

[0, 4, 16, 36, 64]

# 28. 控制推导逻辑的子表达式不要超过两个

In [1]:
# 对维度不多的对象可以使用多阶推导进行拆分，但多过三个的对象还是使用for循环写
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
flat = [x for row in matrix for x in row]
flat

[1, 2, 3, 4, 5, 6, 7, 8, 9]

# 29. 用赋值表达式消除推导中出现的重复代码

In [1]:
stock = {'nails': 125,
         'screws': 35,
         'wingnuts': 8,
         'washers': 24}
order = ['screws', 'wingnuts', 'clips']
def get_batches(count, size = 8):
      return count // size
found = { name: get_batches(stock.get(name, 0)) 
          for name in order if get_batches(stock.get(name, 0))}
found

{'screws': 4, 'wingnuts': 1}

In [2]:
# 使用海豹表达式完成简化
found = {name: batches for name in order
         if (batches := get_batches(stock.get(name, 0)))}
found

{'screws': 4, 'wingnuts': 1}

In [3]:
# 推导顺序很重要，会从条件赋值表达式开始,下式则会报错
result = {name: (tenth := count //10)
          for name, count in stock.items() if tenth >0}

NameError: name 'tenth' is not defined

In [4]:
# 做如下修改就可以正常运行
result = {name: tenth
          for name, count in stock.items() if (tenth := count //10) >0}

In [5]:
result

{'nails': 12, 'screws': 3, 'washers': 2}

In [6]:
# 如果使用了：=赋值，那么在推导中会产生变量泄露
# 即最后一个循环的变量被赋给迭代工具变量（类似for循环）
half = [(last:= count //2) for count in stock.values()]
print(f'last为最后一个值{last}, half列表为{half}')

last为最后一个值12, half列表为[62, 17, 4, 12]


In [7]:
# 如果在推导式中不使用海豹表达式赋值，那么不会产生变量溢出的问题
half = [count //2 for count in stock.values()]
print(half)
print(count)

[62, 17, 4, 12]


NameError: name 'count' is not defined

## 30.不要让函数直接返回列表，应该让他逐渐生成列表的值

In [8]:
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter == ' ':
            result.append(index + 1)
    return result


In [9]:
address = 'Four score and seven years ago...'
result = index_words(address)
print(result[:10])

[0, 5, 11, 15, 21, 27]


- 上述方法有两个缺点
1. 代码杂乱没有突出重点的index,而是突出了append
2. 既要存列表又要return

In [10]:
# 使用生成器的实现
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

In [11]:
[index for index in index_words_iter(address)]

[0, 5, 11, 15, 21, 27]

In [13]:
[index_words_iter(address)]


[<generator object index_words_iter at 0x0000029D9ACA2180>]

In [14]:
it = index_words_iter(address)

In [15]:
# 使用itertools.islice只生成10次
import itertools
itertools.islice(it, 0, 10)

In [16]:
print(list(result))

[0, 5, 11, 15, 21, 27]


## 31. 谨慎地迭代函数所收到的参数

In [18]:
# 定义一个归一化函数
def normalize(numbers: list[int]):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [20]:
vitits = [15, 35, 80]
percentages = normalize(vitits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [22]:
# 生成器/迭代器只能产生一次结果，当嵌套使用时会产生无数据提供的现象。
# 为避免此现象，嵌套时可以只传入生成器的生成函数，在主处理函数内部对生成函数进行实例化
def normalize_func(get_iter):
    total = sum(get_iter())
    reslut = []
    for value in get_iter():
        percent = 100 * value / total
        result.append(percent)
    return result

In [ ]:
# 使用上述函数时，需要传入lambda表达式，这样每次索要get_iter时才能给出一个新生成器
def read_visits(path):
    with open(path) as f:
        for line in f:
            yield int(line)
path = r'./address.txt'
normalize_func(lambda: read_visits(path))

In [24]:
# 可以使用类特性构建一个新容器处理此问题
class ReadVisits:
    def __init__(self, data_path: str) -> None:
        self.data_path = data_path
    
    def __iter__(self):
      with open(self.data_path) as f:
         for line in f:
            yield int(line)
# 如上容器构建后，只需要把新容器传给normalize运行即可。
visits = ReadVisits('./visit.txt')
percentages = normalize(visits)
